<a href="https://colab.research.google.com/github/gregoryGuichard/regressCFD_p/blob/master/learn_multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json
import pandas as pd
import numpy as np
import tensorflow as tf
import math
from sklearn.model_selection import train_test_split

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
import os
os.listdir("drive/My Drive/DEEPCFD4p/data/")

['SP500_data_1h_x.csv',
 'SP500_y_1h_y.csv',
 'FTSE100_data_1h_x.csv',
 'FTSE100_y_1h_y.csv',
 'FTSE100_data_1h_x_norm.csv',
 'FTSE100_y_1h_y_norm.csv',
 'dax30_data_1h_x_norm.csv',
 'dax30_y_1h_y_norm.csv',
 'dax30_x_multiclass.csv',
 'dax30_y_multiclass.csv']

In [0]:
x  = pd.read_csv("drive/My Drive/DEEPCFD4p/data/dax30_x_multiclass.csv")
y = pd.read_csv("drive/My Drive/DEEPCFD4p/data/dax30_y_high.csv")


In [0]:
del x["Unnamed: 0"]
del y["Unnamed: 0"]


In [0]:
#y = pd.get_dummies(y, columns=["0"])

In [0]:
x = np.array(x)
x = x.reshape([x.shape[0], math.ceil(x.shape[1]/22), 22, 1])
y = np.array(y)

fin_train = math.ceil(x.shape[0] * 0.8)


In [0]:
x_train = x[:fin_train]
y_train = y[:fin_train]
y_valid = y[fin_train:-1]
x_valid = x[fin_train:-1]

In [100]:
'''
x_train = x[:fin_train]
y_valid = y[fin_train:]
x_valid = x[fin_train:len(y_valid)]
x_test = x[fin_valid:]


y_train = y[:fin_train]
y_valid = y[fin_train:]
y_test = y[fin_valid:]
'''

'\nx_train = x[:fin_train]\ny_valid = y[fin_train:]\nx_valid = x[fin_train:len(y_valid)]\nx_test = x[fin_valid:]\n\n\ny_train = y[:fin_train]\ny_valid = y[fin_train:]\ny_test = y[fin_valid:]\n'

In [0]:
x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.0001, random_state=42, shuffle=True)

In [0]:
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, Conv2D, MaxPool2D, Flatten
def create_model(x_train):
    model = tf.keras.models.Sequential([
          #bdd dropout
       
        tf.keras.layers.GRU(30, input_shape=[x_train.shape[1], x_train.shape[2]], return_sequences=True,  recurrent_regularizer=tf.keras.regularizers.l2(0.1)),
        tf.keras.layers.GRU(20, input_shape=[x_train.shape[1], x_train.shape[2]], return_sequences=False, recurrent_regularizer=tf.keras.regularizers.l2(0.1)),
        tf.keras.layers.Dense(40, activation="relu"),
        tf.keras.layers.Dense(1)
        #tf.keras.layers.Dense(4, activation="softmax")


    ])
    model.build()
    return model

def build_model(x_train):
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (x_train.shape[1], x_train.shape[2], 1)))
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',  activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    #model.add(Dense(4, activation = "softmax"))

    model.build()
    return model
def train_model(model, x_train, y_train,  x_valid, y_valid):
    early_stopping_monitor = tf.keras.callbacks.EarlyStopping(
    monitor='val_mse',
    #monitor='val_binary_accuracy',
    min_delta=0.001,
    patience=30,
    verbose=1,
    mode='min',
    baseline=None,
    restore_best_weights=True
)
    model.summary()
    model.compile(optimizer="adam",
                  loss="mse",
                  metrics=['mse'])
                  #loss="categorical_crossentropy",
                  #metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=50000, batch_size=32,validation_data=(x_valid, y_valid), validation_freq=1, callbacks=[early_stopping_monitor]
              )

In [0]:
#model = create_model(x_train)
model = build_model(x_train)
train_model(model, x_train, y_train, x_valid, y_valid)

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_44 (Conv2D)           (None, 51, 22, 32)        832       
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 51, 22, 32)        25632     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 25, 11, 32)        0         
_________________________________________________________________
dropout_33 (Dropout)         (None, 25, 11, 32)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 25, 11, 64)        18496     
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 25, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 12, 5, 64)       

In [0]:
#pourquoi pas faire ufe regressiom ?
pred = model.predict(x_valid)

In [0]:
len(pred)

In [0]:
pred = model.predict(x_valid)
tab = {}

for value in np.arange(0, 1, 0.01):
  value = round(value, 2)
  total = 0
  ok =0
  for p, vrai in zip(pred,y_valid):
    if p > value:
      total += 1
      if vrai > 0.2:
        ok +=1
  if total != 0:
    tab[str(value)] = [(ok/total), total]

In [0]:
tab

In [0]:
df_result= pd.DataFrame().from_dict(tab, orient="index")

In [0]:
df_result.plot(y=0)
df_result.plot(y=1)

In [0]:
model_name = "DAX30_high"

In [0]:
model.save_weights(f"drive/My Drive/DEEPCFD4p/model_{model_name}/model")
with open(f"drive/My Drive/DEEPCFD4p/model_{model_name}/tab_short", "w+") as f:
  f.write(json.dumps(tab))